## Исследование: Музыка больших городов
Яндекс.Музыка — это крупный продукт с огромным запасом данных для исследований. Задача проекта: выяснить, как музыка, которая звучит по дороге на работу в понедельник утром, отличается от той, что играет в среду или в конце рабочей недели? Используются данные для Москвы и Петербурга. <br> Сравнить, что и в каком режиме слушают их жители.<br>

# Шаг 1. Получение данных

Изучу данные.

## Импорт библиотек

In [100]:
import pandas as pd 

Прочитаю файл *music_project.csv* и сохраню его в переменной *df*. 

In [101]:
df = pd.read_csv('/datasets/music_project.csv')

Получу первые 10 строк таблицы.

In [102]:
print(df.head(10)) 

     userID                        Track            artist   genre  \
0  FFB692EC            Kamigata To Boots  The Mass Missile    rock   
1  55204538  Delayed Because of Accident  Andreas Rönnberg    rock   
2    20EC38            Funiculì funiculà       Mario Lanza     pop   
3  A3DD03C9        Dragons in the Sunset        Fire + Ice    folk   
4  E2DC1FAE                  Soul People        Space Echo   dance   
5  842029A1                    Преданная         IMPERVTOR  rusrap   
6  4CB90AA5                         True      Roman Messer   dance   
7  F03E1C1F             Feeling This Way   Polina Griffith   dance   
8  8FA1D3BE     И вновь продолжается бой               NaN  ruspop   
9  E772D5C0                    Pessimist               NaN   dance   

             City        time        Day  
0  Saint-Petersburg  20:28:33  Wednesday  
1            Moscow  14:07:09     Friday  
2  Saint-Petersburg  20:58:07  Wednesday  
3  Saint-Petersburg  08:37:09     Monday  
4            M

Общая информация о данных таблицы *df*.




In [103]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


Рассмотрю полученную информацию подробнее.

Всего в таблице 7 столбцов.

Подробно разберу, какие в *df* столбцы и какую информацию они содержат:

* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Количество значений в столбцах различается. Это говорит о том, что в данных есть пропущенные значения.



**Выводы**

Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. <br> Две проблемы, которые нужно решать: пропуски и некачественные названия столбцов. Для проверки рабочих гипотез особенно ценны столбцы *time*, *day* и *City*. Данные из столбца *genre* позволят узнать самые популярные жанры.

# Шаг 2. Предобработка данных

Уберу пропуски, переименую столбцы, а также проверю данные на наличие дубликатов.

Перечень названий столбцов:

In [104]:
print(df.columns) 

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


В названиях столбцов есть пробелы, которые могут затруднять доступ к данным.

Переименую столбцы для удобства дальнейшей работы. Проверю результат.



In [105]:
new_names = ['user_id','track_name','artist_name','genre_name','city', 'time', 'weekday']
df.set_axis(new_names, axis = 'columns', inplace = True) 

In [106]:
print(df.columns) 

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')


Проверю данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [107]:
df.isnull()
print(df.isnull().sum())

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64


Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: скажем,  не назван конкретный исполнитель народной песни. Хуже, если проблемы с записью данных. Каждый отдельный случай необходимо разобрать и выявить причину.

Заменяю пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown'. После этой операции нужно убедиться, что таблица больше не содержит пропусков.

In [108]:
df['track_name'] = df['track_name'].fillna('unknown') 

In [109]:
df['artist_name'] = df['artist_name'].fillna('unknown') 

In [110]:
df.isnull()
print(df.isnull().sum()) 

user_id           0
track_name        0
artist_name       0
genre_name     1198
city              0
time              0
weekday           0
dtype: int64


Удаляю в столбце с жанрами пустые значения; убеждаюсь, что их больше не осталось:

In [111]:
df.dropna(subset = ['genre_name' ], inplace = True) 

In [112]:
df.isnull()
print(df.isnull().sum()) 

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64


Необходимо установить наличие дубликатов.  <br> 
Если найдутся - необходимо удалить, и проверить, все ли удалились.

In [113]:
shape_table = df.shape
print(df.duplicated().sum())

3755


In [114]:
shape_table = df.shape
df = df.drop_duplicates().reset_index(drop=True)

In [115]:
shape_table = df.shape
print(df.duplicated().sum())

0


Дубликаты могли появиться вследствие сбоя в записи данных. <br> Стоит обратить внимание и разобраться с причинами появления такого «информационного мусора».

Сохраняю список уникальных значений столбца с жанрами в переменной *genres_list*. 

Объявляю функцию *find_genre()* для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.





In [116]:
genres_list = df['genre_name'].unique()
print(genres_list)

['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'alternative' 'children' 'rnb' 'hip' 'jazz' 'postrock' 'latin'
 'classical' 'metal' 'reggae' 'tatar' 'blues' 'instrumental' 'rusrock'
 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'shanson' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'contemporary' 'new' 'soul' 'holiday' 'german'
 'tropical' 'fairytail' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'chanson'
 'numetal' 'vocal' 'estrada' 'russian' 'classicmetal' 'dubstep' 'club'
 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french' 'disco'
 'religious' 'hiphop' 'drum' 'extrememetal' 'türkçe' 'experimental' 'easy'
 'metalcore' 'modern' 'argentinetango' 'old' 'breaks' 'eurofolk'
 'stonerrock' 'industrial' 'fu

Cоздание функции find_genre()
Функция принимает как параметр строку с названием искомого жанра, в теле объявляется переменная-счётчик, ей присваивается значение 0, затем цикл for проходит по списку уникальных значений. Если очередной элемент списка равен параметру функции, то значение счётчика увеличивается на 1. По окончании работы цикла функция возвращает значение счётчика.

In [117]:
def find_genre(a):
    i = 0
    for element in genres_list:
        if element == a:
            i += 1
    return i

searsh_genre = find_genre('hip')
print(searsh_genre)

1


Вызов функции *find_genre()* для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — *hiphop*. Поищем другие варианты:

* hip
* hop
* hip-hop


In [118]:
searsh_genre_hip = find_genre('hip')
print(searsh_genre_hip)

1


In [119]:
searsh_genre_hop = find_genre('hop')
print(searsh_genre_hop)

0


In [120]:
searsh_genre_hiphop = find_genre('hip-hop')
print(searsh_genre_hiphop)

0


Объявляю функцию *find_hip_hop()*, которая заменяет  неправильное название этого жанра в столбце *'genre_name'* на *'hiphop'* и проверяет успешность выполнения замены.

Так исправляю все варианты написания, которые выявила проверка.

Создание функции find_hip_hop():
функция принимает как параметры таблицу df и неверное название к столбцу 'genre_name' применяется специальный метод, который заменяет второй параметр на строку 'hiphop'. Результат работы равен подсчитанному методом count() числу значений столбца, которые равны второму параметру, функция возвращает результат.

In [121]:
def find_hip_hop(df, wrong):
    first_value = wrong
    second_value = 'hiphop'
    df['genre_name'] = df['genre_name'].replace(wrong,'hiphop')
    genre_name = df.loc[:, 'genre_name']
    b = genre_name.loc[df[df['genre_name'] == wrong]['genre_name']].count()
    return b
new_value = find_hip_hop(df, 'hip')
print(new_value)

0


Получаю общую информацию о данных. <br> Убеждаюсь, что чистка выполнена успешно.

In [122]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      60126 non-null  object
 1   track_name   60126 non-null  object
 2   artist_name  60126 non-null  object
 3   genre_name   60126 non-null  object
 4   city         60126 non-null  object
 5   time         60126 non-null  object
 6   weekday      60126 non-null  object
dtypes: object(7)
memory usage: 3.2+ MB


**Вывод**

На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и всяческие виды дубликатов. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляю все пропущенные значения, но заполняю пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

# Шаг 3. Исследование данных

# Действительно ли музыку в разных городах слушают по-разному?

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяю это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаю количество прослушанных  в эти дни композиций с известным жанром, и сравниваю результаты.

Группирую данные по городу и вызовом метода *count()* подсчитываю композиции, для которых известен жанр.

In [123]:
group_city = df.groupby('city')['genre_name'].count()
print(group_city)

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64


В Москве прослушиваний больше, чем в Питере, но это не значит, что Москва более активна. У Яндекс.Музыки в целом больше пользователей в Москве, поэтому величины сопоставимы.

Сгруппирую данные по дню недели и подсчитаю прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.

In [124]:
group_weekday = df.groupby('weekday')['genre_name'].count()
print(group_weekday)

weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64


Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.

Создаю функцию *number_tracks()*, которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяю количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

Создание функции number_tracks(): объявляется функция с тремя параметрами: df, day, city, в переменной track_list сохраняются те строки таблицы df, для которых значение в столбце 'weekday' равно параметру day и одновременно значение в столбце 'city' равно параметру city, в переменной track_list_count сохраняется число значений столбца 'genre_name', рассчитанное методом count(), для таблицы track_list функция возвращает значение track_list_count.

In [125]:
def number_tracks(df, day, city):
    track_list = df[(df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count

In [126]:
track_moscow_monday = number_tracks(df, 'Monday', 'Moscow')
print(track_moscow_monday)

15347


In [127]:
track_petersburg_monday = number_tracks(df, 'Monday', 'Saint-Petersburg')
print(track_petersburg_monday)

5519


In [128]:
track_moscow_Wednesday = number_tracks(df, 'Wednesday', 'Moscow')
print(track_moscow_Wednesday)

10865


In [129]:
track_Petersburg_Wednesday = number_tracks(df, 'Wednesday', 'Saint-Petersburg')
print(track_Petersburg_Wednesday)

6913


In [130]:
track_moscow_Friday = number_tracks(df, 'Friday', 'Moscow')
print(track_moscow_Friday)

15680


In [131]:
track_Petersburg_Friday = number_tracks(df, 'Friday', 'Saint-Petersburg')
print(track_Petersburg_Friday)

5802


Сведу полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.


In [132]:
data = [['Moscow', track_moscow_monday, track_moscow_Wednesday, track_moscow_Friday],
       ['Saint-Petersburg', track_petersburg_monday, track_Petersburg_Wednesday, track_Petersburg_Friday]]
columns = ['city','monday','wednesday','friday']
table = pd.DataFrame(data = data, columns = columns)
print(table)

               city  monday  wednesday  friday
0            Moscow   15347      10865   15680
1  Saint-Petersburg    5519       6913    5802


**Вывод**

Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

# Утро понедельника и вечер пятницы — разная музыка или одна и та же?

Отвечу на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).

Получаю таблицы данных по Москве *moscow_general* и по Санкт-Петербургу *spb_general*.

In [133]:
moscow_general = df[(df['city'] == 'Moscow')] 

In [134]:
spb_general = df[(df['city'] == 'Saint-Petersburg')] 

Создаю функцию *genre_weekday()*, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

В переменной genre_list сохраняются те строки df, для которых одновременно:<br> 
1) значение в столбце 'weekday' равно параметру day,<br> 
2) значение в столбце 'time' больше time1 и <br> 
3) меньше time2. <br> 
В переменной genre_list_sorted сохраняются в порядке убывания первые 10 значений Series, полученной подсчётом числа значений 'genre_name', сгруппированной по столбцу 'genre_name' таблицы genre_list. <br> Функция возвращает значение genre_list_sorted.

In [135]:
def genre_weekday(df, day, time1, time2):
    genre_list = df.loc[(df.loc[:,'weekday'] == day) & (df.loc[:,'time'] > time1) & (df.loc[:,'time'] < time2)]
    genre_list_sorted = genre_list.sort_values(by='genre_name', ascending = False).head(10)
    return genre_list_sorted

Cравниваю полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [136]:
genre_weekday(moscow_general, 'Monday','07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
27442,D7FB50DA,Drumming Circle,Professor Trance,worldbeat,Moscow,09:30:47,Monday
20384,F0C5DA9A,Hasta Siempre Comandante,Juan M. Chazarreta,world,Moscow,08:10:09,Monday
36691,C3B36125,Brava la cuoca,Roberto Scaglioni,world,Moscow,09:51:28,Monday
54812,F37EC6D1,Penjereden Dash Gelir,Chingiz Sadykhov,world,Moscow,08:29:31,Monday
58660,59CBCCE5,Dyolmano Dyulbero,The Bulgarian Voices,world,Moscow,09:53:19,Monday
7388,62E1B964,Child of a Blind Man,Hazmat Modine,world,Moscow,09:37:22,Monday
21143,FF3FD2BD,Ire,Adekunle Gold,world,Moscow,08:26:19,Monday
49652,FE3A4C70,Ма царды 'мбал (Спутник жизни),Оксана Джелиева,world,Moscow,08:53:25,Monday
26801,FDF6D919,Nuage,Jim Cole & Spectral Voices,world,Moscow,08:42:04,Monday
40820,3D844B0C,Tango,Athens Symphony Orchestra,world,Moscow,09:45:53,Monday


In [137]:
genre_weekday(spb_general, 'Monday','07:00:00', '11:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
30068,692AEC35,Baiana,Emicida,world,Saint-Petersburg,09:24:38,Monday
42316,6B62E9CE,The Color of The Night,L' Amore Orchestra,world,Saint-Petersburg,08:56:04,Monday
25542,F018BD36,Aicha,Khaled,world,Saint-Petersburg,09:24:54,Monday
25368,888CB0C0,Sichou Zhi Lu,Ethno Music Orchestra,world,Saint-Petersburg,08:49:11,Monday
44109,2DD633B3,Parole parole,unknown,world,Saint-Petersburg,08:50:08,Monday
45091,B5B6FE0A,Sade,Badmos,world,Saint-Petersburg,09:03:38,Monday
21206,F33FA8C,Pivni Tkach posh edit light,Katya Chilly,world,Saint-Petersburg,08:51:27,Monday
20813,4D2A996C,Closer to Heaven,Mysteria,world,Saint-Petersburg,09:30:23,Monday
19541,CA471C2,The Chieri by Avalon Rising,Avalon Rising,world,Saint-Petersburg,09:05:43,Monday
18608,FEA172A3,Jenny Chicken,Kate Mcmayland and Her Orchestra,world,Saint-Petersburg,09:04:32,Monday


In [138]:
genre_weekday(moscow_general, 'Friday','17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
43694,917047E8,The Nomad,Niraj Chag Featuring Melissa Baten,world,Moscow,21:58:43,Friday
30922,3DC9EF93,Aprum Em Keznov,Arabo Ispiryan,world,Moscow,20:07:26,Friday
31820,9F87ADDC,Bir Gozal,unknown,world,Moscow,20:14:37,Friday
52603,4C61D12D,Apple Penne,Bharathvaj,world,Moscow,21:32:05,Friday
31514,5366E775,Die Reblaus,Hans Moser,world,Moscow,21:40:41,Friday
31317,CAAC4375,Kervansaray,Muhammet İrmikçi,world,Moscow,20:22:48,Friday
2878,2487DD6F,Макарий,Абстрактор,world,Moscow,20:57:45,Friday
14375,BCEA335,Yo Soy Maria,Panterinya,world,Moscow,21:11:20,Friday
31013,77BE1010,Kandıramazsın,Genco Ecer,world,Moscow,22:00:25,Friday
52657,6214F479,Forró do Rei,Trio Virgulino,world,Moscow,20:28:05,Friday


In [139]:
genre_weekday(spb_general, 'Friday','17:00:00', '23:00:00')

,user_id,track_name,artist_name,genre_name,city,time,weekday
54324,54A01EAC,Batwanes Beek Ta'a Ninsa,emad batayeh,world,Saint-Petersburg,20:58:18,Friday
29985,47E5088,Что же делать?,Группа «Домбай»,world,Saint-Petersburg,21:15:04,Friday
44531,873D8A1B,Orkestarsko Oro,Maleševski Melos,world,Saint-Petersburg,20:58:55,Friday
18013,1D507F3F,A Cochabamba Me Voy,Daniele Sepe,world,Saint-Petersburg,21:50:57,Friday
48238,F3FB8127,Dari Lolo,Aravod,world,Saint-Petersburg,21:48:19,Friday
22457,A38F488E,The Forest,Taiko Drums,world,Saint-Petersburg,21:27:03,Friday
3470,86F2380D,Королева,Мануэль Исаков,world,Saint-Petersburg,21:19:05,Friday
28527,807DDAF6,El Forga Moura,Haim,world,Saint-Petersburg,21:11:00,Friday
38883,BF27088D,Let It Out - Rafijah & Niyo Rah,Rafijah & Niyo Rah,world,Saint-Petersburg,21:28:56,Friday
29561,DAE710B3,Les nuits de patrimonio,Jérôme Ciosi,world,Saint-Petersburg,20:07:58,Friday


Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр *world*.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр *world*.

**Вывод**

Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

# Москва и Питер — две разные столицы, два разных направления в музыке. Правда?

Гипотеза: Питер богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает попсу.



Сгруппирую таблицу *moscow_general* по жанру, сосчитаю численность композиций каждого жанра методом *count()*, отсортирую в порядке убывания и сохраню результат в таблице *moscow_genres*.

Просмотрю первые 10 строк этой новой таблицы.

In [140]:
moscow_genres = moscow_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [141]:
print(moscow_genres.head(10))

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre_name, dtype: int64


Сгруппирую таблицу *spb_general* по жанру, сосчитаю численность композиций каждого жанра методом *count()*, отсортирую в порядке убывания и сохраню результат в таблице *spb_genres*.

Просмотрю первые 10 строк этой таблицы. Теперь можно сравнивать два города.

In [142]:
spb_genres = spb_general.groupby('genre_name')['genre_name'].count().sort_values(ascending = False)

In [143]:
print(spb_genres.head(10))

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre_name, dtype: int64


**Вывод**

В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

# Шаг 4. Результаты исследования


Рабочие гипотезы:

* музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;

* списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;

* население двух городов предпочитает разные музыкальные жанры.

**Общие результаты**

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза подтверждена, вторая гипотеза подтверждена и третья - не подтверждена.